In [1]:
using FFTW, Random, Test

include("MacroUtils.jl")
using .MacroUtils: cleanup

In [2]:
Random.seed!(1234)
sample_vector_input = rand(32);
sample_tuple_input = Tuple(sample_vector_input);

In [3]:
function test_fft(fft)
    @test FFTW.fft(sample_vector_input) ≈ fft(sample_tuple_input)
end

test_fft (generic function with 1 method)

In [4]:
function fft_rec_c(a, p, b, m)
    n = length(p)
    isone(n) && (b[1] = a[p[1]]; return m)
    n2 = n ÷ 2
    w = exp(-2 * pi * im / n)

    p1 = [p[k] for k in 1:2:n]
    p2 = [p[k] for k in 2:2:n]
    y1 = Vector{Complex{Float64}}(undef, n2)
    y2 = Vector{Complex{Float64}}(undef, n2)
    m += 1
    m = fft_rec_c(a, p1, y1, m)
    m = fft_rec_c(a, p2, y2, m)
    for k in 1:n2
        t = w^(k - 1) * y2[k]
        b[k] = y1[k] + t
        b[k+n2] = y1[k] - t
    end
    return m
end

fft_rec_c (generic function with 1 method)

In [5]:
function fft_rec_c(a)
    n = length(a)
    @assert ispow2(n)
    p = [k for k in 1:n]
    b = Vector{Complex{Float64}}(undef, n)
    m = fft_rec_c(a, p, b, 1)
    return b
end

fft_rec_c (generic function with 2 methods)

In [6]:
fft_rec_c((10.0, 20.0, 30.0, 40.0))

4-element Vector{ComplexF64}:
 100.0 + 0.0im
 -20.0 + 20.0im
 -20.0 + 0.0im
 -20.0 - 20.0im

In [7]:
test_fft(fft_rec_c)

Test Passed

In [8]:
mult_r(x_r::T, x_i::T, y_r::T, y_i::T) where {T} =
    x_r * y_r - x_i * y_i
mult_i(x_r::T, x_i::T, y_r::T, y_i::T) where {T} =
    x_r * y_i + x_i * y_r

mult_i (generic function with 1 method)

In [9]:
function fft_rec(a, p, b_r, b_i, m)
    n = length(p)
    isone(n) && begin
        b_r[1] = a[p[1]]
        b_i[1] = 0.0
        return m
    end
    n2 = n ÷ 2
    w = exp(-2 * pi * im / n)

    p1 = [p[k] for k in 1:2:n]
    p2 = [p[k] for k in 2:2:n]
    y1_r = Vector{Float64}(undef, n2)
    y1_i = Vector{Float64}(undef, n2)
    y2_r = Vector{Float64}(undef, n2)
    y2_i = Vector{Float64}(undef, n2)
    m += 1
    m = fft_rec(a, p1, y1_r, y1_i, m)
    m = fft_rec(a, p2, y2_r, y2_i, m)
    for k in 1:n2
        wk = w^(k - 1)
        t_r = mult_r(real(wk), imag(wk), y2_r[k], y2_i[k])
        t_i = mult_i(real(wk), imag(wk), y2_r[k], y2_i[k])
        b_r[k] = y1_r[k] + t_r
        b_i[k] = y1_i[k] + t_i
        b_r[k+n2] = y1_r[k] - t_r
        b_i[k+n2] = y1_i[k] - t_i
    end
    return m
end

fft_rec (generic function with 1 method)

In [10]:
function fft_rec(a)
    n = length(a)
    @assert ispow2(n)
    p = [k for k in 1:n]
    b_r = Vector{Float64}(undef, n)
    b_i = Vector{Float64}(undef, n)
    m = fft_rec(a, p, b_r, b_i, 1)
    return [complex(b_r[k], b_i[k]) for k in 1:n]
end

fft_rec (generic function with 2 methods)

In [11]:
fft_rec((10.0, 20.0, 30.0, 40.0))

4-element Vector{ComplexF64}:
 100.0 + 0.0im
 -20.0 + 20.0im
 -20.0 + 0.0im
 -20.0 - 20.0im

In [12]:
test_fft(fft_rec)

Test Passed

In [13]:
mult_r_gen1(x_r, x_i, y_r, y_i) =
    :($x_r * $y_r - $x_i * $y_i)
mult_i_gen1(x_r, x_i, y_r, y_i) =
    :($x_r * $y_i + $x_i * $y_r)

mult_i_gen1 (generic function with 1 method)

In [14]:
function fft_gen_impl1(p, b_r, b_i, m)
    n = length(p)
    es = Expr[]

    if isone(n)
        append!(es, (
            :($(b_r[1]) = a[$(p[1])]),
            :($(b_i[1]) = 0.0)
        ))
        return es, m
    end

    n2 = n ÷ 2
    p1 = [p[k] for k in 1:2:n]
    p2 = [p[k] for k in 2:2:n]

    y1_r = [Symbol("y1_r_", m, "_", k) for k in 1:n2]
    y1_i = [Symbol("y1_i_", m, "_", k) for k in 1:n2]
    y2_r = [Symbol("y2_r_", m, "_", k) for k in 1:n2]
    y2_i = [Symbol("y2_i_", m, "_", k) for k in 1:n2]
    m += 1

    es1, m = fft_gen_impl1(p1, y1_r, y1_i, m)
    es2, m = fft_gen_impl1(p2, y2_r, y2_i, m)
    append!(es, es1, es2)

    w = exp(-2 * pi * im / n)

    for k in 1:n2
        wk = w^(k - 1)
        t_r = Symbol("t_r_", m)
        t_i = Symbol("t_i_", m)
        m += 1

        append!(es, (
            :($t_r = $(mult_r_gen1(real(wk), imag(wk), y2_r[k], y2_i[k]))),
            :($t_i = $(mult_i_gen1(real(wk), imag(wk), y2_r[k], y2_i[k]))),
            :($(b_r[k]) = $(y1_r[k]) + $t_r),
            :($(b_i[k]) = $(y1_i[k]) + $t_i),
            :($(b_r[k+n2]) = $(y1_r[k]) - $t_r),
            :($(b_i[k+n2]) = $(y1_i[k]) - $(t_i))
        ))
    end


    return es, m
end

fft_gen_impl1 (generic function with 1 method)

In [15]:
function fft_gen_impl1(n)
    @assert (n >= 1 && ispow2(n))

    p = [k for k in 1:n]
    b_r = [Symbol("y0_r_", k) for k in 1:n]
    b_i = [Symbol("y0_i_", k) for k in 1:n]
    es, m = fft_gen_impl1(p, b_r, b_i, 1)

    b = [:(complex($(b_r[k]), $(b_i[k]))) for k in 1:n]
    quote
        $(es...)
        return $(Expr(:vect, b...))
    end

end

fft_gen_impl1 (generic function with 2 methods)

In [16]:
fft_gen_impl1(4) |> cleanup

quote
    y1_r_2_1 = a[1]
    y1_i_2_1 = 0.0
    y2_r_2_1 = a[3]
    y2_i_2_1 = 0.0
    t_r_3 = 1.0y2_r_2_1 - 0.0y2_i_2_1
    t_i_3 = 1.0y2_i_2_1 + 0.0y2_r_2_1
    y1_r_1_1 = y1_r_2_1 + t_r_3
    y1_i_1_1 = y1_i_2_1 + t_i_3
    y1_r_1_2 = y1_r_2_1 - t_r_3
    y1_i_1_2 = y1_i_2_1 - t_i_3
    y1_r_4_1 = a[2]
    y1_i_4_1 = 0.0
    y2_r_4_1 = a[4]
    y2_i_4_1 = 0.0
    t_r_5 = 1.0y2_r_4_1 - 0.0y2_i_4_1
    t_i_5 = 1.0y2_i_4_1 + 0.0y2_r_4_1
    y2_r_1_1 = y1_r_4_1 + t_r_5
    y2_i_1_1 = y1_i_4_1 + t_i_5
    y2_r_1_2 = y1_r_4_1 - t_r_5
    y2_i_1_2 = y1_i_4_1 - t_i_5
    t_r_6 = 1.0y2_r_1_1 - 0.0y2_i_1_1
    t_i_6 = 1.0y2_i_1_1 + 0.0y2_r_1_1
    y0_r_1 = y1_r_1_1 + t_r_6
    y0_i_1 = y1_i_1_1 + t_i_6
    y0_r_3 = y1_r_1_1 - t_r_6
    y0_i_3 = y1_i_1_1 - t_i_6
    t_r_7 = 6.123233995736766e-17y2_r_1_2 - -1.0y2_i_1_2
    t_i_7 = 6.123233995736766e-17y2_i_1_2 + -1.0y2_r_1_2
    y0_r_2 = y1_r_1_2 + t_r_7
    y0_i_2 = y1_i_1_2 + t_i_7
    y0_r_4 = y1_r_1_2 - t_r_7
    y0_i_4 = y1_i_1_2 - t_i_7


In [17]:
@generated function fft_gen1(a::NTuple{N,Float64}) where {N}
    fft_gen_impl1(N)
end

fft_gen1 (generic function with 1 method)

In [18]:
fft_gen1((10.0, 20.0, 30.0, 40.0))

4-element Vector{ComplexF64}:
 100.0 + 0.0im
 -20.0 + 20.0im
 -20.0 + 0.0im
 -20.0 - 20.0im

In [19]:
test_fft(fft_gen1)

Test Passed

Абстрактная интерпретация (метавычисления).

In [20]:
abstract type MVal end

struct MCst <: MVal
    v::Float64
end

struct MAny <: MVal
    v
end

In [21]:
Base.hash(x::MCst, h::UInt) = hash(x.v, hash(MCst, h))
Base.hash(x::MAny, h::UInt) = hash(x.v, hash(MAny, h))

Base.:(==)(x::MVal, y::MVal) = x.v == y.v

In [22]:
add_s(x::MVal, y::MVal) = MAny(:($(x.v) + $(y.v)))

add_s(x::MCst, y::MCst) = MCst(x.v + y.v)

function add_s(x::MCst, y::MAny)
    if x.v == 0.0
        y
    else
        MAny(:($(x.v) + $(y.v)))
    end
end

function add_s(x::MAny, y::MCst)
    if y.v == 0.0
        x
    else
        MAny(:($(x.v) + $(y.v)))
    end
end

add_s (generic function with 4 methods)

In [23]:
sub_m(x::MVal, y::MVal) = MAny(:($(x.v) - $(y.v)))

sub_m(x::MCst, y::MCst) = MCst(x.v - y.v)

function sub_m(x::MCst, y::MAny)
    if x.v == 0.0
        MAny(:(- $(y.v)))
    else
        MAny(:($(x.v) - $(y.v)))
    end
end

function sub_m(x::MAny, y::MCst)
    if y.v == 0.0
        x
    else
        MAny(:($(x.v) - $(y.v)))
    end
end

sub_m (generic function with 4 methods)

In [24]:
mult_m(x::MVal, y::MVal) = MAny(:($(x.v) * $(y.v)))

mult_m(x::MCst, y::MCst) = MCst(:($(x.v * y.v)))

function mult_m(x::MCst, y::MAny)
    if x.v == 0.0
        MCst(0.0)
    elseif x.v == 1.0
        y
    elseif x.v == -1.0
        MAny(:(-$(y.v)))
    else
        MAny(:($(x.v) * $(y.v)))
    end
end

function mult_m(x::MAny, y::MCst)
    if y.v == 0.0
        MCst(0.0)
    elseif y.v == 1.0
        x
    elseif y.v == -1.0
        MAny(:(-$(x.v)))
    else
        MAny(:($(x.v) * $(y.v)))
    end
end

mult_m (generic function with 4 methods)

In [25]:
mult_m(MCst(10.0), MAny(:x))

MAny(:(10.0x))

In [26]:
@test mult_m(MCst(2.0), MCst(3.0)) == MCst(6.0)
@test mult_m(MCst(10.0), MAny(:x)) == MAny(:(10.0 * x))
@test mult_m(MCst(0.0), MAny(:x)) == MCst(0.0)
@test mult_m(MCst(1.0), MAny(:x)) == MAny(:x)
@test mult_m(MCst(-1.0), MAny(:x)) == MAny(:(- x))
@test mult_m(MAny(:x), MCst(20.0)) == MAny(:(x * 20.0))
@test mult_m(MAny(:x), MCst(0.0)) == MCst(0.0)
@test mult_m(MAny(:x), MCst(1.0)) == MAny(:x)
@test mult_m(MAny(:x), MCst(-1.0)) == MAny(:(-x))
@test mult_m(MAny(:x), MAny(:y)) == MAny(:(x * y))

Test Passed

In [27]:
mult_r_g(x_r, x_i, y_r, y_i) =
    sub_m(mult_m(x_r, y_r), mult_m(x_i, y_i))

mult_r_g (generic function with 1 method)

In [28]:
mult_i_g(x_r, x_i, y_r, y_i) =
    add_m(mult_m(x_r, y_i), mult_m(x_i, y_r))

mult_i_g (generic function with 1 method)

In [29]:
g(::Symbol, x::MCst) = x
g(s::Symbol, ::MAny) = MAny(s)

g (generic function with 2 methods)

In [30]:
function fft_gen_impl2(p, b_r, b_i, d, m)
    n = length(p)
    es = Expr[]

    if isone(n)
        d[b_r[1]] = MAny(:(a[$(p[1])]))
        d[b_i[1]] = MCst(0.0)
        append!(es, (
            :($(b_r[1]) = $(d[b_r[1]].v)),
            :($(b_i[1]) = $(d[b_i[1]].v))
        ))
        return es, m
    end

    n2 = n ÷ 2
    p1 = [p[k] for k in 1:2:n]
    p2 = [p[k] for k in 2:2:n]

    y1_r = [Symbol("y1_r_", m, "_", k) for k in 1:n2]
    y1_i = [Symbol("y1_i_", m, "_", k) for k in 1:n2]
    y2_r = [Symbol("y2_r_", m, "_", k) for k in 1:n2]
    y2_i = [Symbol("y2_i_", m, "_", k) for k in 1:n2]
    m += 1

    es1, m = fft_gen_impl2(p1, y1_r, y1_i, d, m)
    es2, m = fft_gen_impl2(p2, y2_r, y2_i, d, m)
    append!(es, es1, es2)

    w = exp(-2 * pi * im / n)

    for k in 1:n2
        wk = w^(k - 1)
        t_r = Symbol("t_r_", m)
        t_i = Symbol("t_i_", m)
        m += 1

        t_r_m = mult_r_g(MCst(real(wk)), MCst(imag(wk)), d[y2_r[k]], d[y2_i[k]])
        t_i_m = mult_i_g(MCst(real(wk)), MCst(imag(wk)), d[y2_r[k]], d[y2_i[k]])
        b_r1_m = add_m(d[y1_r[k]], g(t_r, t_r_m))
        d[b_r[k]] = g(b_r[k], b_r1_m)
        b_i1_m = add_m(d[y1_i[k]], g(t_i, t_i_m))
        d[b_i[k]] = g(b_i[k], b_i1_m)
        b_r2_m = sub_m(d[y1_r[k]], g(t_r, t_r_m))
        d[b_r[k+n2]] = g(b_r[k+n2], b_r2_m)
        b_i2_m = sub_m(d[y1_i[k]], g(t_i, t_i_m))
        d[b_i[k+n2]] = g(b_i[k+n2], b_i2_m)

        append!(es, (
            :($t_r = $(t_r_m.v)),
            :($t_i = $(t_i_m.v)),
            :($(b_r[k]) = $(b_r1_m.v)),
            :($(b_i[k]) = $(b_i1_m.v)),
            :($(b_r[k+n2]) = $(b_r2_m.v)),
            :($(b_i[k+n2]) = $(b_i2_m.v))
        ))
    end

    return es, m
end

fft_gen_impl2 (generic function with 1 method)

In [31]:
function fft_gen_impl2(n)
    @assert (n >= 1 && ispow2(n))

    p = [k for k in 1:n]
    b_r = [Symbol("y0_r_", k) for k in 1:n]
    b_i = [Symbol("y0_i_", k) for k in 1:n]
    d = Dict{Symbol,MVal}()
    es, m = fft_gen_impl2(p, b_r, b_i, d, 1)

    b = [:(complex($(b_r[k]), $(b_i[k]))) for k in 1:n]
    # println(d)
    quote
        $(es...)
        return $(Expr(:vect, b...))
    end

end

fft_gen_impl2 (generic function with 2 methods)

In [32]:
fft_gen_impl2(4) |> cleanup

UndefVarError: UndefVarError: `add_m` not defined

In [33]:
@generated function fft_gen2(a::NTuple{N,Float64}) where {N}
    fft_gen_impl2(N)
end

fft_gen2 (generic function with 1 method)

In [34]:
fft_gen2((10.0, 20.0, 30.0, 40.0))

UndefVarError: UndefVarError: `add_m` not defined

In [35]:
test_fft(fft_gen2)

Error During Test at /home/roman/devel/Julia/julia-samples/CompTimeSamples/jupyter/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W2sZmlsZQ==.jl:2
  Test threw exception
  Expression: FFTW.fft(sample_vector_input) ≈ fft(sample_tuple_input)
  UndefVarError: `add_m` not defined
  Stacktrace:
   [1] mult_i_g(x_r::MCst, x_i::MCst, y_r::MAny, y_i::MCst)
     @ Main ~/devel/Julia/julia-samples/CompTimeSamples/jupyter/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X41sZmlsZQ==.jl:1
   [2] fft_gen_impl2(p::Vector{Int64}, b_r::Vector{Symbol}, b_i::Vector{Symbol}, d::Dict{Symbol, MVal}, m::Int64)
     @ Main ~/devel/Julia/julia-samples/CompTimeSamples/jupyter/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X43sZmlsZQ==.jl:38
   [3] fft_gen_impl2(p::Vector{Int64}, b_r::Vector{Symbol}, b_i::Vector{Symbol}, d::Dict{Symbol, MVal}, m::Int64) (repeats 4 times)
     @ Main ~/devel/Julia/julia-samples/CompTimeSamples/jupyter/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X43sZmlsZQ==.jl:25
  

Test.FallbackTestSetException: Test.FallbackTestSetException("There was an error during testing")

## Без `MCst` и `MAny`

Можно различать константы и без `MCst` и `MAny`.

Но при этом (чуть-чуть) нарушается принцип, что "нельзя заглядывать в генерируемый код".

In [36]:
add_s(x, y) = :($x + $y)

add_s(x::Float64, y::Float64) = :($(x + y))

function add_s(x::Float64, y)
    if x == 0.0
        y
    else
        :($x + $y)
    end
end

function add_s(x, y::Float64)
    if y == 0.0
        x
    else
        :($x + $y)
    end
end

add_s (generic function with 8 methods)

In [37]:
@test add_s(:(2.0), :(3.0)) == :(5.0)
@test add_s(:(2.0), :(x)) == :(2.0 + x)
@test add_s(:(0.0), :(x)) == :(x)
@test add_s(:(x), :(2.0)) == :(x + 2.0)
@test add_s(:(x), :(0.0)) == :(x)
@test add_s(:(x), :(y)) == :(x + y)

Test Passed

In [38]:
sub_s(x, y) = :($x - $y)

sub_s(x::Float64, y::Float64) = :($(x - y))

function sub_s(x::Float64, y)
    if x == 0.0
        :(- $y)
    else
        :($x - $y)
    end
end

function sub_s(x, y::Float64)
    if y == 0.0
        x
    else
        :($x - $y)
    end
end

sub_s (generic function with 4 methods)

In [39]:
@test sub_s(:(5.0), :(2.0)) == :(3.0)
@test sub_s(:(2.0), :(x)) == :(2.0 - x)
@test sub_s(:(0.0), :(x)) == :(-x)
@test sub_s(:(x), :(2.0)) == :(x - 2.0)
@test sub_s(:(x), :(0.0)) == :(x)
@test sub_s(:(x), :(y)) == :(x - y)

Test Passed

In [40]:
mult_s(x, y) = :($x * $y)

mult_s(x::Float64, y::Float64) = :($(x * y))

function mult_s(x::Float64, y)
    if x == 0.0
        :(0.0)
    elseif x == 1.0
        y
    elseif x == -1.0
        :(-$y)
    else
        :($x * $y)
    end
end

function mult_s(x, y::Float64)
    if y == 0.0
        :(0.0)
    elseif y == 1.0
        x
    elseif y == -1.0
        :(-$x)
    else
        :($x * $y)
    end
end

mult_s (generic function with 4 methods)

In [41]:
@test mult_s(2.0, 3.0) == 6.0
@test mult_s(10.0, :x) ==:(10.0 * x)
@test mult_s(0.0, :x) == 0.0
@test mult_s(1.0, :x) == :x
@test mult_s(-1.0, :x) == :(- x)
@test mult_s(:x, 20.0) == :(x * 20.0)
@test mult_s(:x, 0.0) == 0.0
@test mult_s(:x, 1.0) == :x
@test mult_s(:x, -1.0) == :(-x)
@test mult_s(:x, :y) == :(x * y)

Test Passed

In [42]:
mult_r_s(x_r, x_i, y_r, y_i) =
    sub_s(mult_s(x_r, y_r), mult_s(x_i, y_i))
mult_i_s(x_r, x_i, y_r, y_i) =
    add_s(mult_s(x_r, y_i), mult_s(x_i, y_r))

mult_i_s (generic function with 1 method)

In [43]:
h(::Symbol, x::Float64) = x
h(s::Symbol, e) = s

h (generic function with 2 methods)

In [44]:
function fft_gen_impl3(p, b_r, b_i, d, m)
    n = length(p)
    es = Expr[]

    if isone(n)
        # d[b_r[1]] = :(a[$(p[1])])
        d[b_r[1]] = b_r[1]
        d[b_i[1]] = 0.0
        append!(es, (
            # :($(b_r[1]) = $(d[b_r[1]])),
            # :($(b_i[1]) = $(d[b_i[1]]))
            :($(b_r[1]) = a[$(p[1])]),
            :($(b_i[1]) = 0.0)
            ))
        return es, m
    end

    n2 = n ÷ 2
    p1 = [p[k] for k in 1:2:n]
    p2 = [p[k] for k in 2:2:n]

    y1_r = [Symbol("y1_r_", m, "_", k) for k in 1:n2]
    y1_i = [Symbol("y1_i_", m, "_", k) for k in 1:n2]
    y2_r = [Symbol("y2_r_", m, "_", k) for k in 1:n2]
    y2_i = [Symbol("y2_i_", m, "_", k) for k in 1:n2]
    m += 1

    es1, m = fft_gen_impl3(p1, y1_r, y1_i, d, m)
    es2, m = fft_gen_impl3(p2, y2_r, y2_i, d, m)
    append!(es, es1, es2)

    w = exp(-2 * pi * im / n)

    for k in 1:n2
        wk = w^(k - 1)
        t_r = Symbol("t_r_", m)
        t_i = Symbol("t_i_", m)
        m += 1

        t_r_m = mult_r_s(real(wk), imag(wk), d[y2_r[k]], d[y2_i[k]])
        t_i_m = mult_i_s(real(wk), imag(wk), d[y2_r[k]], d[y2_i[k]])
        b_r1_m = add_s(d[y1_r[k]], h(t_r, t_r_m))
        d[b_r[k]] = h(b_r[k], b_r1_m)
        b_i1_m = add_s(d[y1_i[k]], h(t_i, t_i_m))
        d[b_i[k]] = h(b_i[k], b_i1_m)
        b_r2_m = sub_s(d[y1_r[k]], h(t_r, t_r_m))
        d[b_r[k+n2]] = h(b_r[k+n2], b_r2_m)
        b_i2_m = sub_s(d[y1_i[k]], h(t_i, t_i_m))
        d[b_i[k+n2]] = h(b_i[k+n2], b_i2_m)

        append!(es, (
            :($t_r = $t_r_m),
            :($t_i = $t_i_m),
            :($(b_r[k]) = $b_r1_m),
            :($(b_i[k]) = $b_i1_m),
            :($(b_r[k+n2]) = $b_r2_m),
            :($(b_i[k+n2]) = $b_i2_m)
        ))
    end

    return es, m
end

fft_gen_impl3 (generic function with 1 method)

In [45]:
function fft_gen_impl3(n)
    @assert (n >= 1 && ispow2(n))

    p = [k for k in 1:n]
    b_r = [Symbol("y0_r_", k) for k in 1:n]
    b_i = [Symbol("y0_i_", k) for k in 1:n]
    d = Dict{Symbol,Any}()
    es, m = fft_gen_impl3(p, b_r, b_i, d, 1)

    b = [:(complex($(b_r[k]), $(b_i[k]))) for k in 1:n]
    # println(d)
    quote
        $(es...)
        return $(Expr(:vect, b...))
    end

end

fft_gen_impl3 (generic function with 2 methods)

In [46]:
fft_gen_impl3(4) |> cleanup

quote
    y1_r_2_1 = a[1]
    y1_i_2_1 = 0.0
    y2_r_2_1 = a[3]
    y2_i_2_1 = 0.0
    t_r_3 = y2_r_2_1
    t_i_3 = 0.0
    y1_r_1_1 = y1_r_2_1 + t_r_3
    y1_i_1_1 = 0.0
    y1_r_1_2 = y1_r_2_1 - t_r_3
    y1_i_1_2 = 0.0
    y1_r_4_1 = a[2]
    y1_i_4_1 = 0.0
    y2_r_4_1 = a[4]
    y2_i_4_1 = 0.0
    t_r_5 = y2_r_4_1
    t_i_5 = 0.0
    y2_r_1_1 = y1_r_4_1 + t_r_5
    y2_i_1_1 = 0.0
    y2_r_1_2 = y1_r_4_1 - t_r_5
    y2_i_1_2 = 0.0
    t_r_6 = y2_r_1_1
    t_i_6 = 0.0
    y0_r_1 = y1_r_1_1 + t_r_6
    y0_i_1 = 0.0
    y0_r_3 = y1_r_1_1 - t_r_6
    y0_i_3 = 0.0
    t_r_7 = 6.123233995736766e-17y2_r_1_2
    t_i_7 = -y2_r_1_2
    y0_r_2 = y1_r_1_2 + t_r_7
    y0_i_2 = t_i_7
    y0_r_4 = y1_r_1_2 - t_r_7
    y0_i_4 = -t_i_7
    return [complex(y0_r_1, y0_i_1), complex(y0_r_2, y0_i_2), complex(y0_r_3, y0_i_3), complex(y0_r_4, y0_i_4)]
end

In [47]:
@generated function fft_gen2(a::NTuple{N,Float64}) where {N}
    fft_gen_impl3(N)
end

fft_gen2 (generic function with 1 method)

In [48]:
fft_gen2((10.0, 20.0, 30.0, 40.0))

4-element Vector{ComplexF64}:
 100.0 + 0.0im
 -20.0 + 20.0im
 -20.0 + 0.0im
 -20.0 - 20.0im

In [49]:
test_fft(fft_gen2)

Test Passed